In [1]:
from pyspark.sql import SparkSession

spark_session = SparkSession\
        .builder\
        .master("spark://130.238.28.137:7077") \
        .appName("markus_lindgren_A3")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.cores.max", 4)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

24/03/11 16:04:04 WARN Utils: Your hostname, markus-HP-Laptop-15s-eq0xxx resolves to a loopback address: 127.0.1.1; using 130.243.144.87 instead (on interface wlo1)
24/03/11 16:04:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 16:04:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark_context

<SparkContext master=spark://130.238.28.137:7077 appName=markus_lindgren_A3>

In [20]:
data = spark_session.read.json("hdfs://130.238.28.181:9000/MyData/corpus-webis-tldr-17.json")
data.printSchema()

Py4JJavaError: An error occurred while calling o138.json.
: java.net.ConnectException: Call From markus-HP-Laptop-15s-eq0xxx/127.0.1.1 to 130.238.28.181:9000 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at java.base/jdk.internal.reflect.GeneratedConstructorAccessor56.newInstance(Unknown Source)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:913)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:828)
	at org.apache.hadoop.ipc.Client.getRpcResponse(Client.java:1616)
	at org.apache.hadoop.ipc.Client.call(Client.java:1558)
	at org.apache.hadoop.ipc.Client.call(Client.java:1455)
	at org.apache.hadoop.ipc.ProtobufRpcEngine2$Invoker.invoke(ProtobufRpcEngine2.j